# Как распарсить вал и трейн сеты

Можете парсить как нравится, я просто подумала так будет удобнее

In [1]:
import json
from func_to_call import parse_all_data, parse_data_with_time


data_v1 = parse_all_data('datasets/val_set.json')

data_v2 = parse_data_with_time('datasets/val_set.json')

with open('parsed_tuning.json', 'w', encoding='utf-8') as f:
    json.dump(data_v1, f, ensure_ascii=False)

with open('parsed_dash.json', 'w', encoding='utf-8') as f:
    json.dump(data_v2, f, ensure_ascii=False)

In [2]:
import json

with open('parsed_tuning.json', 'r', encoding='utf-8') as f:
    training_data = json.load(f)

In [3]:
formatted_data = []

for item in training_data:
    contexts = "\n".join([ctx['text'] for ctx in item['contexts']])
    base_input = f"Вопрос: {item['user_question']}\nКонтекст: {contexts}"

    if item['winner'] == 'Saiga':
        formatted_data.append({
            "input": base_input,
            "output": item['saiga_answer'],
            "source": "saiga",
            "rating": "good" if item['winner'] in ['Saiga', 'Оба хорошо'] else "bad"
        })
        
    elif item['winner'] == 'GigaChat':
        formatted_data.append({
            "input": base_input,
            "output": item['giga_answer'],
            "source": "giga",
            "rating": "good" if item['winner'] in ['GigaChat', 'Оба хорошо'] else "bad"
        })
        
    elif item['winner'] == 'Оба хорошо':
        formatted_data.extend([
            {
                "input": base_input,
                "output": item['saiga_answer'],
                "source": "saiga",
                "rating": "good"
            },
            {
                "input": base_input,
                "output": item['giga_answer'],
                "source": "giga",
                "rating": "good"
            }
        ])
        
    elif item['winner'] == 'Оба плохо':
        formatted_data.extend([
            {
                "input": base_input,
                "output": item['saiga_answer'],
                "source": "saiga",
                "rating": "bad"
            },
            {
                "input": base_input,
                "output": item['giga_answer'],
                "source": "giga",
                "rating": "bad"
            }
        ])
        
    else:
        formatted_data.append({
            "input": base_input,
            "output": item['saiga_answer'],
            "source": "unknown",
            "rating": "neutral"
        })

In [5]:
formatted_data[0]

{'input': 'Вопрос: Чем заняться вечером, если не хочется сидеть дома?\nКонтекст: ающихся) В соответствии с частью 4 статьи 34 Федерального закона от 29.12.2012 №273-ФЗ «Об образовании в Российской Федерации» НИУ ВШЭ обеспечивает обучающимся реализацию их права на посещение по своему выбору мероприятий, которые проводятся в университете и не предусмотрены учебным планом. К числу мероприятий, не предусмотренных учебным планом, относятся праздники, конкурсы, тематические вечера, фестивали, спортивные соревнования, выставки и другие мероприятия, порядок проведения которых не установлен иными локальными нормативными актами НИУ ВШЭ. Мероприятия, не предусмотренные учебным планом, проводятся в целях: 1.4.1. личностного развития обучающихся, формирования профессиональной культуры будущих специалистов, их гражданской позиции, мировоззрения и социальной активности; 1.4.2. реализации гражданско-патриотического, эстетического, духовного физического воспитания и профессиональной ориентации обучающи

# Формат того как должен выглядеть датасет чтобы метрики посчитать

In [ ]:
import pandas as pd

dataset = pd.DataFrame({
    "question": [
        "Какие документы регулируют порядок обслуживания студентов в столовой?",
        "Каковы основные этапы прохождения учебной практики?",
    ],

    "answer": [
        "Я ГЛУПИ МОДЕЛЬ НЕ УМЕЮ...",
        "А Я УМЕЮ...", #ответы вашей модели
    ],
    "ground_truth": [
        "Порядок организации обслуживания регулируется следующими документами...",
        "Основные этапы включают: 1. Размещение программы...", #эталонные ответы
    ],
    "contexts": [
        [
            "сотрудника НИУ ВШЭ с указанием руководителя...",
            "Центр сервиса «Студент» – Национальный исследовательский университет...",
            # остальные контексты для первого вопроса
        ],
        [
            "уточной аттестации и текущего контроля успеваемости студентов...",
            "Траектории обучения в бакалавриате...",
            # ... контексты для второго вопроса
        ],
    ]
})

В файле с метриками prepocess_calculate/metrics.py указано описание каждой метрики


In [ ]:
from metrics import ValidatorSimple

vs = ValidatorSimple(neural=True)
vs.validate_rag(dataset)